In [96]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
%matplotlib inline
import codecs
import re
import enchant

In [149]:
import os
import sys

In [193]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import time
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC

In [134]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from word2vecUtils import utils
from nltk.stem import SnowballStemmer

In [5]:
filename = "./Data/data_segments2.json"
data = open(filename).read()

In [32]:
#pattern = re.compile(r'{[\n\s]*"_id"[a-zA-Z0-9~`!@#$%^&*()\-_+={}\[\]:;\"\'<>,.?/|\\]*')
#arr = pattern.findall(data)

In [36]:
def load_json_multiple(segments):
    chunk = ""
    for segment in segments:
        chunk += segment
        try:
            yield json.loads(chunk)
            chunk = ""
        except ValueError:
            pass

In [38]:
data = []
with open(filename) as f:
    for parsed_json in load_json_multiple(f):
        data.append(parsed_json)
print len(data)

113


In [48]:
data1 = []
for i in xrange(len(data)):
    for j in data[i]["transcription_merged"]:
        temp = j
        temp["_id"] = data[i]["_id"]
        data1.append(temp)

In [71]:
df = pd.DataFrame(data1)
df.head()

,_id,conv_no,end_time,line,question,segment,speaker,start_time
0,d7da99f7-013b-4d63-84d9-421226ee5249,0.0,30,at indian east india company now by the way wh...,NaN,NaN,Rep,0
1,d7da99f7-013b-4d63-84d9-421226ee5249,1.0,31,HMM.,NaN,NaN,Prospect,30
2,d7da99f7-013b-4d63-84d9-421226ee5249,2.0,34,so you're creating tons of synonyms on the bac...,NaN,NaN,Rep,31
3,d7da99f7-013b-4d63-84d9-421226ee5249,3.0,35,it is manual.,NaN,NaN,Prospect,34
4,d7da99f7-013b-4d63-84d9-421226ee5249,4.0,53,right exactly you got it then next comes is th...,NaN,NaN,Rep,35


In [55]:
df[df.segment=="None"].shape

(2390, 8)

In [72]:
df[~df.segment.isnull()].shape

(4612, 8)

In [52]:
df.segment.value_counts()

None                           2390
Rep questions                   663
Customer questions              523
Next steps and action items     228
Client reference                198
Pricing                         114
Pitch                           110
Agenda                           89
Customer pain points             88
Objection                        65
Competitor differentiation       46
Relevant case study              32
Brilio pitch                     32
Competitor                       16
Closing discussion               13
Next steps                        2
Unbxd pitch                       2
WebEngage pitch                   1
Name: segment, dtype: int64

In [73]:
df1 = df[~df.segment.isnull()]

In [74]:
df1.head()

,_id,conv_no,end_time,line,question,segment,speaker,start_time
11,d7da99f7-013b-4d63-84d9-421226ee5249,11.0,245,the next piece what I really wanted to also hi...,NaN,Pitch,Rep,87
17,d7da99f7-013b-4d63-84d9-421226ee5249,17.0,303,however what Celeb rose is not very good at is...,NaN,Competitor differentiation,Rep,260
19,d7da99f7-013b-4d63-84d9-421226ee5249,19.0,364,yes yes that is interesting to me now one one ...,NaN,Customer pain points,Prospect,318
21,d7da99f7-013b-4d63-84d9-421226ee5249,21.0,407,so we have and I did this for a couple of reas...,NaN,Customer pain points,Prospect,365
24,d7da99f7-013b-4d63-84d9-421226ee5249,24.0,429,you guys offer a link search in navigation.,NaN,Customer questions,Prospect,426


In [ ]:
df1["target"] = np.where(df1["segment"]=="Next steps and action items",1,0)

In [80]:
df1.end_time = df1.end_time.astype(np.float64)
df1.start_time = df1.start_time.astype(np.float64)
df1.conv_no = df1.conv_no.astype(np.float64)

In [81]:
df1 = df1.set_index(np.arange(len(df1)))

In [82]:
df1.head()

,_id,conv_no,end_time,line,question,segment,speaker,start_time
0,d7da99f7-013b-4d63-84d9-421226ee5249,11.0,245.0,the next piece what I really wanted to also hi...,NaN,Pitch,Rep,87.0
1,d7da99f7-013b-4d63-84d9-421226ee5249,17.0,303.0,however what Celeb rose is not very good at is...,NaN,Competitor differentiation,Rep,260.0
2,d7da99f7-013b-4d63-84d9-421226ee5249,19.0,364.0,yes yes that is interesting to me now one one ...,NaN,Customer pain points,Prospect,318.0
3,d7da99f7-013b-4d63-84d9-421226ee5249,21.0,407.0,so we have and I did this for a couple of reas...,NaN,Customer pain points,Prospect,365.0
4,d7da99f7-013b-4d63-84d9-421226ee5249,24.0,429.0,you guys offer a link search in navigation.,NaN,Customer questions,Prospect,426.0


In [84]:
df1["time_difference"] = df1["end_time"] - df1["start_time"]
df1["time_difference"] = df1["time_difference"]/max(df1["time_difference"])
df1["conv_no"] = df1["conv_no"] / max(df1["conv_no"])

In [86]:
df1.speaker.value_counts()

Rep         2818
Prospect    1794
Name: speaker, dtype: int64

In [88]:
le = LabelEncoder()
df1["speaker"] = le.fit_transform(df1["speaker"])

In [92]:
df1.shape

(4612, 9)

In [94]:
df1 = df1.drop(["_id","question"],axis=1)

In [ ]:
dict = enchant.Dict("en_US")

In [99]:
#df1["line"] = df1["line"].apply(lambda x: re.sub(r"[^\x00-\x7f]"," ",x)
#df1["line"] = df1["line"].apply(lambda x: re.sub(r"^[a-zA-Z0-9\s]", "", x))

In [138]:
#df1["line"] = df1["line"].apply(str)
df1["line"] = df1["line"].apply(lambda x: re.sub(r"[^\x00-\x7f]"," ",x))

In [111]:
df1["line"] = df1["line"].apply(str)

In [113]:
df1["line"] = df1["line"].apply(lambda x: re.sub(r"[\s]+"," ", x).strip())
df1["line"] = df1["line"].apply(lambda x: x.lower())

In [116]:
analyzer = SentimentIntensityAnalyzer()

In [122]:
df1['compoundScore'] = df1['line'].apply(lambda x: analyzer.polarity_scores(x)["compound"])
df1['negSent'] = df1['line'].apply(lambda x: analyzer.polarity_scores(x)["neg"])
df1['posSent'] = df1['line'].apply(lambda x: analyzer.polarity_scores(x)["pos"])
df1['neuSent'] = df1['line'].apply(lambda x: analyzer.polarity_scores(x)["neu"])

In [125]:
stop_words = set(stopwords.words('english'))

In [128]:
df1["line"] = df1["line"].apply(lambda x: " ".join([i for i in word_tokenize(x) if i not in stop_words]))

In [135]:
stemmer = SnowballStemmer('english')

In [139]:
df1["line"] = df1["line"].apply(lambda x: " ".join([stemmer.stem(i) for i in word_tokenize(x)]))

In [141]:
df1["line"] = df1["line"].apply(lambda x: " ".join([i for i in word_tokenize(x) if len(x)>2]))

In [142]:
df1.head()

,conv_no,end_time,line,segment,speaker,start_time,time_difference,compoundScore,negSent,posSent,neuSent
0,0.029650,245.0,next piec realli want also highlight coupl thi...,Pitch,1,87.0,0.335456,0.9971,0.018,0.204,0.778
1,0.045822,303.0,owev celeb rose good < > spellcheck n't know g...,Competitor differentiation,1,260.0,0.091295,0.7039,0.018,0.077,0.905
2,0.051213,364.0,es yes interest one one one question thing wan...,Customer pain points,0,318.0,0.097665,0.9509,0.000,0.163,0.837
3,0.056604,407.0,coupl reason one well first thing get influenc...,Customer pain points,0,365.0,0.089172,0.2500,0.032,0.053,0.915
4,0.064690,429.0,ou guy offer link search navig .,Customer questions,0,426.0,0.006369,0.0000,0.000,0.000,1.000


In [150]:
clean_desc = []
for index, row in df1.iterrows():
    clean_desc.append(" ".join(
        utils.review_to_wordlist(row["line"], False)))

vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor = None,
                             stop_words = "english",
                             max_features = 3000)

data_features = vectorizer.fit_transform(clean_desc)
np.asarray(data_features)
data_features = data_features.astype(np.float32)
features_df = pd.DataFrame(data_features.todense(), columns=vectorizer.get_feature_names())

In [151]:
features_df.shape

(4612, 3000)

In [152]:
df1 = pd.concat([df1, features_df], axis=1)

In [154]:
df1 = df1.drop(["start_time","end_time"],axis=1)

In [161]:
df1 = df1.drop(["line","segment"],axis=1)

In [162]:
train = df1.sample(frac=0.8,random_state=211)
test = df1.drop(train.index)

In [163]:
y_train = train["target"]
y_test = test["target"]
X_train = train.drop(["target"],axis=1)
X_test = test.drop(["target"],axis=1)

In [164]:
gbm = lgb.LGBMClassifier(n_estimators=3000, max_depth=3, subsample=0.7, colsample_bytree= 0.7)
gbm = gbm.fit(X_train, y_train)

In [188]:
y_pred = gbm.predict_proba(X_test)

In [189]:
y_pred

array([[  9.99082292e-01,   8.85479835e-04,   3.22285569e-05],
       [  9.99840524e-01,   1.46293847e-04,   1.31820306e-05],
       [  9.99720408e-01,   2.74870572e-04,   4.72132789e-06],
       ..., 
       [  9.97216883e-01,   2.45843813e-03,   3.24678593e-04],
       [  9.99437219e-01,   4.98783467e-04,   6.39973732e-05],
       [  9.99512944e-01,   4.63790575e-04,   2.32655399e-05]])

In [177]:
y_result = []
magic = 0.64
for i in range(0, len(X_test)):
    if y_pred[i][0] > magic:
        y_result.append(0)
    else:
        y_result.append(1)

In [180]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [179]:
accuracy_score(y_test,y_result)

0.99240780911062909

In [185]:
print metrics.classification_report(y_test,y_result)

             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00       915
        1.0       0.00      0.00      0.00         6
        2.0       0.00      0.00      0.00         1

avg / total       0.98      0.99      0.99       922



In [186]:
np.savetxt('lgb',y_pred,delimiter = ',', fmt = '%0.6f')

In [190]:
y_result

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [191]:
sum(y_test)

8.0

In [192]:
sum(y_result)

0